<a href="https://colab.research.google.com/github/SushSushmithagowda/ML-101-projects/blob/main/Graph_neural_network_implementation_with_Protein_benchmark_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, I have carried out two implementations of GCNs to classify the PROTEINS benchmark dataset.

Approach 1: GCN with Spektral

In [ ]:
!pip install numpy
!pip install tensorflow
!pip install spektral

import numpy as np
import tensorflow as tf
import spektral

In [ ]:
#reading in the PROTEINS dataset
from spektral.datasets import TUDataset


In [ ]:
#Spectral provides the TUDataset class, which contains benchmark datasets for graph classification
data = TUDataset('PROTEINS')

100%|█████████████████████████████████████████| 447k/447k [00:00<00:00, 689kB/s]


Successfully loaded PROTEINS.


In [ ]:
data

TUDataset(n_graphs=1113)

In [ ]:
# Since we want to utilize the Spektral GCN layer, we want to follow the original paper for this method and perform some preprocessing:
from spektral.transforms import GCNFilter

data.apply(GCNFilter())

In [ ]:
# Split our train and test data. This just splits based on the first 80%/second 20% which isn't entirely ideal, so we'll shuffle the data first.
import numpy as np
np.random.shuffle(data)
split = int(0.8 * len(data))
data_train, data_test = data[:split], data[split:]

In [ ]:
# Spektral is built on top of Keras, so we can use the Keras functional API to build a model that first embeds,
# then sums the nodes together (global pooling), then classifies the result with a dense softmax layer

# First, let's import the necessary layers:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GlobalSumPool

In [ ]:
# Now, we can use model subclassing to define our model:
class ProteinsGNN(Model):

  def __init__(self, n_hidden, n_labels):
    super().__init__()
    # Define our GCN layer with our n_hidden layers
    self.graph_conv = GCNConv(n_hidden)
    # Define our global pooling layer
    self.pool = GlobalSumPool()
    # Define our dropout layer, initialize dropout freq. to .5 (50%)
    self.dropout = Dropout(0.5)
    # Define our Dense layer, with softmax activation function
    self.dense = Dense(n_labels, 'softmax')

# Define class method to call model on input
  def call(self, inputs):
    out = self.graph_conv(inputs)
    out = self.dropout(out)
    out = self.pool(out)
    out = self.dense(out)

    return out

In [ ]:
# Instantiate our model for training
model = ProteinsGNN(32, data.n_labels)

In [ ]:
# Compile model with our optimizer (adam) and loss function
model.compile('adam', 'categorical_crossentropy')

In [ ]:
# Here's the trick - we can't just call Keras' fit() method on this model.
# Instead, we have to use Loaders, which Spektral walks us through. Loaders create mini-batches by iterating over the graph
# Since we're using Spektral for an experiment, for our first trial we'll use the recommended loader in the getting started tutorial

# TODO: read up on modes and try other loaders later
from spektral.data import BatchLoader

loader = BatchLoader(data_train, batch_size=32)

In [ ]:
# Now we can train! We don't need to specify a batch size, since our loader is basically a generator
# But we do need to specify the steps_per_epoch parameter

model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=10)

Epoch 1/10
28/28 [==============================] - 5s 55ms/step - loss: 29.2422
Epoch 2/10
28/28 [==============================] - 1s 35ms/step - loss: 9.5231
Epoch 3/10
28/28 [==============================] - 1s 31ms/step - loss: 6.4763
Epoch 4/10
28/28 [==============================] - 1s 28ms/step - loss: 6.9496
Epoch 5/10
28/28 [==============================] - 1s 27ms/step - loss: 6.8298
Epoch 6/10
28/28 [==============================] - 1s 25ms/step - loss: 5.6849
Epoch 7/10
28/28 [==============================] - 1s 24ms/step - loss: 5.5387
Epoch 8/10
28/28 [==============================] - 1s 28ms/step - loss: 6.2545
Epoch 9/10
28/28 [==============================] - 1s 25ms/step - loss: 5.1228
Epoch 10/10
28/28 [==============================] - 1s 28ms/step - loss: 6.0772


In [ ]:
# To evaluate, let's instantiate another loader to test

test_loader = BatchLoader(data_test, batch_size=32)

In [ ]:
# And feed it to our model by calling .load()

loss = model.evaluate(loader.load(), steps=loader.steps_per_epoch)

print('Test loss: {}'.format(loss))

28/28 [==============================] - 1s 21ms/step - loss: 3.8005
Test loss: 3.8004508018493652


This concludes building GCN's with Spektral.

Approach - 2
GCN's with Pytorch-Geometric

In [ ]:
# Install required packages.
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric

     |████████████████████████████████| 48 kB 3.3 MB/s 
     |████████████████████████████████| 407 kB 8.4 MB/s 


In [ ]:
import torch
from torch_geometric.datasets import TUDataset

# Like Spektral, pytorch geometric provides us with benchmark TUDatasets
dataset = TUDataset(root='data/TUDataset', name='PROTEINS')

Extracting data/TUDataset/PROTEINS/PROTEINS.zip
Processing...
Done!


In [ ]:
# Let's take a look at our data. We'll look at dataset (all data) and data (our first graph):

data = dataset[0]  # Get the first graph object.

print()
print(f'Dataset: {dataset}:')
print('====================')
# How many graphs?
print(f'Number of graphs: {len(dataset)}')
# How many features?
print(f'Number of features: {dataset.num_features}')
# Now, in our first graph, how many edges?
print(f'Number of edges: {data.num_edges}')
# Average node degree?
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
# Do we have isolated nodes?
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
# Do we contain self-loops?
print(f'Contains self-loops: {data.contains_self_loops()}')
# Is this an undirected graph?
print(f'Is undirected: {data.is_undirected()}')


Dataset: PROTEINS(1113):
Number of graphs: 1113
Number of features: 3
Number of edges: 162
Average node degree: 3.86
Contains isolated nodes: False
Contains self-loops: False
Is undirected: True


/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'contains_isolated_nodes' is deprecated, use 'has_isolated_nodes' instead
  warnings.warn(out)
/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'contains_self_loops' is deprecated, use 'has_self_loops' instead
  warnings.warn(out)


In [ ]:
# Now, we need to perform our train/test split.
# We create a seed, and then shuffle our data
torch.manual_seed(12345)
dataset = dataset.shuffle()

# Once it's shuffled, we slice the data to split
train_dataset = dataset[150:-150]
test_dataset = dataset[0:150]

# Take a look at the training versus test graphs
print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 813
Number of test graphs: 150


In [ ]:
# Import DataLoader for batching
from torch_geometric.data import DataLoader

# our DataLoader creates diagonal adjacency matrices, and concatenates features
# and target matrices in the node dimension. This allows differing numbers of nodes and edges 
# over examples in one batch. (from pytorch geometric docs)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
# Import everything we need to build our network:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

# Define our GCN class as a pytorch Module
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        # We inherit from pytorch geometric's GCN class, and we initialize three layers
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        # Our final linear layer will define our output
        self.lin = Linear(hidden_channels, dataset.num_classes)
        
    def forward(self, x, edge_index, batch):
      # 1. Obtain node embeddings 
      x = self.conv1(x, edge_index)
      x = x.relu()
      x = self.conv2(x, edge_index)
      x = x.relu()
      x = self.conv3(x, edge_index)
      
      # 2. Readout layer
      x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

      # 3. Apply a final classifier
      x = F.dropout(x, p=0.5, training=self.training)
      x = self.lin(x)
      return x
    
model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(3, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [ ]:
# Set our optimizer (adam)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# Define our loss function
criterion = torch.nn.CrossEntropyLoss()

# Initialize our train function
def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
      out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
      loss = criterion(out, data.y)  # Compute the loss.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      optimizer.zero_grad()  # Clear gradients.
      
# Define our test function
def test(loader):
  model.eval()

  correct = 0
  for data in loader:  # Iterate in batches over the training/test dataset.
      out = model(data.x, data.edge_index, data.batch)  
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      correct += int((pred == data.y).sum())  # Check against ground-truth labels.
  return correct / len(loader.dataset)  # Derive ratio of correct predictions.


# Run for 200 epochs (range is exclusive in the upper bound)
for epoch in range(1, 201):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Train Acc: 0.6335, Test Acc: 0.6267
Epoch: 002, Train Acc: 0.6248, Test Acc: 0.6867
Epoch: 003, Train Acc: 0.6347, Test Acc: 0.6267
Epoch: 004, Train Acc: 0.6494, Test Acc: 0.6800
Epoch: 005, Train Acc: 0.6913, Test Acc: 0.7467
Epoch: 006, Train Acc: 0.6802, Test Acc: 0.7333
Epoch: 007, Train Acc: 0.7109, Test Acc: 0.7200
Epoch: 008, Train Acc: 0.7146, Test Acc: 0.7267
Epoch: 009, Train Acc: 0.6827, Test Acc: 0.7133
Epoch: 010, Train Acc: 0.6753, Test Acc: 0.6667
Epoch: 011, Train Acc: 0.6421, Test Acc: 0.6267
Epoch: 012, Train Acc: 0.6667, Test Acc: 0.6467
Epoch: 013, Train Acc: 0.6642, Test Acc: 0.6933
Epoch: 014, Train Acc: 0.6986, Test Acc: 0.7067
Epoch: 015, Train Acc: 0.7220, Test Acc: 0.7267
Epoch: 016, Train Acc: 0.6581, Test Acc: 0.7067
Epoch: 017, Train Acc: 0.6814, Test Acc: 0.7067
Epoch: 018, Train Acc: 0.6716, Test Acc: 0.6867
Epoch: 019, Train Acc: 0.6679, Test Acc: 0.6667
Epoch: 020, Train Acc: 0.6851, Test Acc: 0.6867
Epoch: 021, Train Acc: 0.7159, Test Acc:

Looking at the last 20 epochs, the model seems to have achieved 70% training accuracy and about 76% testing accuracy. This infers the model being able to predict good results having low bias without inclining towards 'overfitting','underfitting' pitfall.   